In [1]:
import numpy as np
import pandas as pd
import math
import keras
import keras.backend as K
import theano.tensor as T
from keras.models import load_model
from keras.models import Sequential
from multiprocessing import Pool
from functools import partial
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD, RMSprop,Adam
from datetime import datetime
import sys
import time
from sklearn import preprocessing
from sklearn import metrics

import copy
from sklearn.metrics import average_precision_score
from keras.initializers import Initializer
from keras import initializers

from sklearn import preprocessing

path="/axp/rim/imsadsml/warehouse/sagra39/gimhack/"

Using TensorFlow backend.


In [2]:
train=pd.read_csv(path+'train_final11_e_available.csv')
print(train.shape)

FileNotFoundError: File b'/axp/rim/imsadsml/warehouse/sagra39/gimhack/train_final11_e_available.csv' does not exist

In [ ]:
valid = pd.read_csv(path+'valid_final11_e.csv')
print(valid.shape)

In [4]:
train =train.replace(['-999',-999],0)
valid= valid.replace([-999,'-999'],0)

In [5]:
ls_train = ['roc_key','departure_date','charge_date']
ls_score = ['condition', 'departure_date','charge_date']
train_new = train.drop(ls_train,axis=1)
valid_new = valid.drop(ls_score,axis=1)

In [6]:
train_scaled = preprocessing.StandardScaler().fit_transform(train_new.values)
train_scaled = pd.DataFrame(train_scaled)

valid_scaled = preprocessing.StandardScaler().fit_transform(valid_new.values)
valid_scaled = pd.DataFrame(valid_scaled)

In [7]:
train_x = train_scaled.drop([164],axis=1)
train_y= train_scaled[164]
valid_x = valid_scaled

In [8]:
class myfunc(Initializer):
    """Initializer that generates tensors with a uniform distribution.
    # Arguments
        minval: A python scalar or a scalar tensor. Lower bound of the range
          of random values to generate.
        maxval: A python scalar or a scalar tensor. Upper bound of the range
          of random values to generate.  Defaults to 1 for float types.
        seed: A Python integer. Used to seed the random generator.
    """

    def __init__(self, minval=0, seed=None,maxval=1):
        self.minval = minval
        self.maxval = maxval
        self.seed = seed

    def __call__(self,shape, dtype=None):
        return (0.1/np.sqrt(shape[0] + 1))*(2*K.random_uniform(shape, self.minval, self.maxval,dtype=dtype,seed=self.seed)-1)

    def get_config(self):
        return {
            'minval': self.minval,
            'maxval': self.maxval,
            'seed': self.seed,
        }



In [9]:
input_dim = len(train_x.columns)
print(input_dim)
hidden_dim = 40

164


In [10]:
def mse_error(y_true, y_pred):
        K.logical_not = lambda x: 1 - x
        return K.mean(K.switch(K.logical_not((y_true==0)),K.square(y_pred-y_true),0),axis=-1)


In [ ]:
model     = Sequential()
model.add(Dense(hidden_dim, input_dim=input_dim, activation='relu',kernel_initializer=myfunc(minval=0,seed=1337)))
model.add(Dense(1, activation='linear',kernel_initializer=myfunc(minval=0,seed=1337)))

In [ ]:
sgd = Adam(lr=0.0001,decay=0.0)
model.compile(loss=mse_error, optimizer=sgd)
model.fit(train_x.values,train_y.values, epochs =10, batch_size=50,shuffle=False, verbose = 0)

In [ ]:
score_condition = valid[['condition','charge_date']].reset_index(drop=True)
pred_score = pd.DataFrame(model.predict(valid_x.values),columns=['score']).reset_index(drop=True)
final_data = pd.concat([score_condition,pred_score],axis=1)
final_data_new = final_data.sort(["condition","score"]).groupby("condition", as_index=False).first()
final_data_new

In [ ]:
score_condition = valid[['condition','charge_date']].reset_index(drop=True)
pred_score = pd.DataFrame(model.predict(valid_x.values),columns=['score']).reset_index(drop=True)
final_data = pd.concat([score_condition,pred_score],axis=1)
final_data_new = final_data.sort(["condition","score"]).groupby("condition", as_index=False).first()
final_data_new